In [8]:
import asyncio
import websockets
import json
import pandas as pd
from datetime import datetime, timezone

apiKeys = pd.read_csv("/Users/maier-borst.h/Documents/apiKeys.csv", sep=";")
apiKeyAIS = apiKeys[apiKeys["Service"] == "AISStream"]["key"].iloc[0]


In [14]:
async def connect_ais_stream():
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {
            "APIKey": apiKeyAIS,  # Required !
            "BoundingBoxes": [
          [
            [
              30.40921772695401,
                31.844734608073395
            ],
            [
              11.603304207355961,
                              44.75088774620332
            ]
          ]
        ],  # Required!
            "FilterMessageTypes": ["PositionReport"]  # Optional!
        }

        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)

        try:
            await asyncio.wait_for(process_messages(websocket), timeout=120)
        except asyncio.TimeoutError:
            print("Timeout occurred. Stopping execution.")

async def process_messages(websocket):
    async for message_json in websocket:
        message = json.loads(message_json)
        message_type = message["MessageType"]
        websocket.close()

        if message_type == "PositionReport":
            ais_message = message['Message']['PositionReport']
            print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Longitude: {ais_message['Longitude']}")

# Call the function
#asyncio.run(connect_ais_stream())

In [15]:
await connect_ais_stream()

Timeout occurred. Stopping execution.
